In [ ]:
# Importing the necessary libraries
from bs4 import BeautifulSoup as bs 
import requests

In [ ]:
#Loading a webpage so we can locate the infobox
r = requests.get("https://en.wikipedia.org/wiki/Aston_Villa_F.C.")

# Converting it to a beautiful soup object
soup = bs(r.content)

# Prin out the HTML
contents = soup.prettify()
print(contents) 

In [ ]:
# We only want info from the infobox shown on the upper rightside on every wikipedia page -> check using inspecting
# Inspecting shows table is stored under 'infobox vcard'

info_box = soup.find(class_="infobox vcard")
print(info_box.prettify())



In [ ]:
info_box = soup.find(class_="infobox vcard")
info_rows = info_box.find_all("tr")
for row in info_rows:
    print(row.prettify())


In [ ]:
# following loop is created to return the football clubs name based on the syntax:
# <tr>
# <th class="infobox-label" scope="row" style="white-space: nowrap; text-align:left">
# Full name
# </th>
# <td class="infobox-data">
# Aston Villa Football Club
# </td>
# </tr>
# In simple terms this loop looks up the class info label with the string full name, if it finds this it will proceed to search
# for the next element whith the class infobox-data, in which holds the actual name.
# if the td element is found it then proceeds to, extract the text, strip it form white space and store it in the key of club_info


club_info = {}

for row in info_rows:
    th_element = th_element = row.find("th", class_="infobox-label", string="Full name")
    if th_element:
        td_element = row.find_next("td", class_="infobox-data")
        if td_element:
            club_info['title'] = td_element.get_text(strip=True)
            break  # Exit the loop once the club name is found

print(club_info.get('title', 'Club name not found'))


In [ ]:
club_info = {}

for row in info_rows:
    # Find the <th> element (table header)
    th_element = row.find("th", class_="infobox-label")
    
    # Find the corresponding <td> element (table data)
    td_element = row.find_next("td", class_="infobox-data")
    
    # Check if both <th> and <td> elements are found
    if th_element and td_element:
        # Extract text from the elements
        key = th_element.get_text(strip=True)
        value = td_element.get_text(strip=True)
        
        # Add key-value pair to the dictionary
        club_info[key] = value

print(club_info)


In [ ]:
import re

club_info = {}

for row in info_rows:
    th_element = row.find("th", class_="infobox-label")
    td_element = row.find_next("td", class_="infobox-data")
    
    if th_element and td_element:
        key = th_element.get_text(strip=True)
        value = td_element.get_text(strip=True)
        
        if key == 'Founded':
            # Use regular expression to extract only the year
            match = re.search(r'\b(\d{4})\b', value)
            if match:
                club_info[key] = match.group(1)
        else:
            club_info[key] = value

print(club_info)


In [ ]:
#### Get info box for all teams that ever played in premier leauge

r = requests.get("https://en.wikipedia.org/wiki/List_of_Premier_League_clubs")
# Converting it to a beautiful soup object
soup = bs(r.content)

# Prin out the HTML
contents = soup.prettify()
print(contents)

In [ ]:
clubs = soup.select(".wikitable.sortable :is(i, b, :not([style]))")
clubs


In [ ]:
club_link = soup.select_one(".wikitable.sortable a[title^='']")['href']
club_link

In [ ]:
club_links = []

for club in soup.select(".wikitable.sortable a[title^='']"):
    club_links.append(club['href'])

print(club_links)


In [ ]:
club_info = {}

for row in soup.select(".wikitable tbody tr"):
    club_name_element = row.select_one("td a[title^='']")
    if club_name_element:
        club_name = club_name_element.get_text(strip=True)
        club_link = club_name_element['href']
        
        club_info[club_name] = club_link

print(club_info)


In [ ]:
def get_info_box(url):
    
    r = requests.get(url)
    
    soup = bs(r.content)
    contents = soup.prettify(r)
    info_box = soup.find(class_="infobox vcard")
    info_rows = info_box.find_all("tr")
    
    import re

    club_info = {}

    for row in info_rows:
        th_element = row.find("th", class_="infobox-label")
        td_element = row.find_next("td", class_="infobox-data")
    
            if th_element and td_element:
            key = th_element.get_text(strip=True)
            value = td_element.get_text(strip=True)
        
                if key == 'Founded':
                # Use regular expression to extract only the year
                match = re.search(r'\b(\d{4})\b', value)
                if match:
                club_info[key] = match.group(1)
                else:
                club_info[key] = value

    return club_info


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import re
import json

club_info = {}

# First code to scrape URLs
r = requests.get("https://en.wikipedia.org/wiki/List_of_Premier_League_clubs")
soup = bs(r.content)

for row in soup.select(".wikitable tbody tr"):
    club_name_element = row.select_one("td a[title^='']")
    if club_name_element:
        club_name = club_name_element.get_text(strip=True)
        club_link = club_name_element['href']
        
        club_info[club_name] = club_link

# List to store all club data
all_club_data = []

# Second code to scrape infobox using the URLs
for club_name, club_link in club_info.items():
    url = f"https://en.wikipedia.org{club_link}"
    r = requests.get(url)
    soup = bs(r.content)

    info_rows = soup.select(".infobox tbody tr")
    club_data = {}

    for row in info_rows:
        th_element = row.find("th", class_="infobox-label")
        td_element = row.find_next("td", class_="infobox-data")

        if th_element and td_element:
            key = th_element.get_text(strip=True)
            value = td_element.get_text(strip=True)

            if key == 'Founded':
                match = re.search(r'\b(\d{4})\b', value)
                if match:
                    club_data[key] = match.group(1)
            else:
                club_data[key] = value

    # Append club data to the list
    all_club_data.append({"Club": club_name, "Data": club_data})

# Save the list to a JSON file
save_data("PremierL_data.json", all_club_data)


In [ ]:
#### Save/reload Clubs Data

import json

def save_data(title, data):
    with open(title,'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent =2)

In [ ]:
import json

def load_data(title):
    with open(title, encoding="utf-8") as f:
        return json.load(f)

In [ ]:
save_data("PremierL_data.json", all_club_data )

In [ ]:
print(all_club_data)

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import json

club_info = {}

# First code to scrape URLs
r = requests.get("https://en.wikipedia.org/wiki/List_of_Premier_League_clubs")
soup = bs(r.content)

for row in soup.select(".wikitable tbody tr"):
    club_name_element = row.select_one("td a[title^='']")
    if club_name_element:
        club_name = club_name_element.get_text(strip=True)
        club_link = club_name_element['href']

        club_info[club_name] = club_link

# List to store all club data
all_club_data = []

# Second code to scrape infobox using the URLs
for club_name, club_link in club_info.items():
    url = f"https://en.wikipedia.org{club_link}"
    r = requests.get(url)
    soup = bs(r.content)

    info_rows = soup.select(".infobox tbody tr")
    club_data = {}

    for row in info_rows:
        th_element = row.find("th", class_="infobox-label")
        td_element = row.find_next("td", class_="infobox-data")

        if th_element and td_element:
            key = th_element.get_text(strip=True)

            # Check if the key is "Nickname(s)" or "Owners"
            if key == 'Nickname(s)' or key == 'Owners':
                items = list(td_element.stripped_strings)
                club_data[key] = items
            else:
                value = td_element.get_text(strip=True)
                if key == 'Founded':
                    match = re.search(r'\b(\d{4})\b', value)
                    if match:
                        club_data[key] = match.group(1)
                else:
                    club_data[key] = value

    # Append club data to the list
    all_club_data.append({"Club": club_name, "Data": club_data})

# Save the list to a JSON file
with open("PremierL_data.json", "w") as json_file:
    json.dump(all_club_data, json_file, indent=2)
